# Exercise 4: Neural Networks Learning

Hand-written digits recognition with (part 1) multi-class logistic regression and (part 2) neural networks. The digits are a subset of the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) by Yann LeCun.

Files provided by Coursera, located under `../exercises/ex1-ex8-octave/ex4`

- `ex4.m` - Octave/MATLAB script that steps you through the exercise
- `ex4data1.mat` - Training set of hand-written digits
- `ex4weights.mat` - Initial weights for the neural network
- `submit.m` - Submission script that sends your solutions to our servers
- `displayData.m` - Function to help visualize the dataset
- `fmincg.m` - Function minimization routine (similar to `fminunc`)
- `sigmoid.m` - Sigmoid function
- `computeNumericalGradient.m` - Numerically compute gradients
- `checkNNGradients.m` - Function to help check your gradients
- `debugInitializeWeights.m` - Function for initializing weights
- `predict.m` - Neural network prediction function


Files to complete:

- `sigmoidGradient.m` - Compute the gradient of the sigmoid function
- `randInitializeWeights.m` - Randomly initialize weights
- `nnCostFunction.m` - Neural network cost function
- `predict.m` - Neural network prediction function

Workflow:

- Download latest Octave version of exercise from Coursera
- Complete code in exercise files following `ex4.pdf`
- Whenever an exercise part is finished
  - Check it with `ex4` in Octave terminal
  - Create a submission token on Coursera (exercise submission page, it lasts 30 minutes)
  - Execute `submit` in Octave terminal
  - Introduce email and token
  - Results appear

**Overview of contents:**

0. Setup: `gnuplot`
1. A

